In [89]:
import os
import numpy as np

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#!{sys.executable} -m pip install bayesian_optimization
from bayes_opt import BayesianOptimization

# SETTINGS

np.random.seed(124)
np.set_printoptions(precision=14)

test_size = 0.1


In [90]:
def get_data():

    ## train & test paths
    data_dir = "./data/"

    ### csv w/ layout: "y,x1,x2,x3,x4,x5"
    train_file = "train.csv"
    data_path = data_dir + train_file
    
    data_type = np.double # np dtype object for values in X and Y

    X = np.genfromtxt(data_path,
                        dtype = data_type,
                        delimiter = ",",
                        skip_header = 1,
                        usecols = np.arange(2, 5 + 2))

    y = np.genfromtxt(data_path,
                        dtype = data_type,
                        delimiter = ",",
                        skip_header = 1,
                        usecols = np.arange(1, 1 + 1))

    return X, y

In [91]:
def save_submission(submission):

    ## prediction submission paths
    sub_dir = "./submission/"
    sub_file = "submission.csv"
    sub_path = sub_dir + sub_file


    if not os.path.isdir(sub_dir):
        os.mkdir(sub_dir)


    ## submission csv header
    sub_header = ""
    sub_format = "%.13f"
    np.savetxt(sub_path,
               submission,
               fmt = sub_format,
               delimiter = ',',
               header = sub_header,
               comments = '')

In [92]:

def get_phi(X):
    phi_1_5 = lambda x: x
    phi_6_10 = lambda x: np.square(x)
    phi_11_15 = lambda x: np.exp(x)
    phi_16_20 = lambda x: np.cos(x)
    phi_21 = lambda x: np.array([1])

    phi = []
    for x in X:
        phi_x = np.concatenate([phi_(x) for phi_ in [phi_1_5, phi_6_10, phi_11_15, phi_16_20, phi_21]])
        # print(phi_x)
        phi.append(phi_x)

    return phi

In [93]:
X, y = get_data()
phi = get_phi(X)
print(phi[5])

X_train, X_test, y_train, y_test = train_test_split(phi, y, test_size = test_size, random_state=124)



[-0.05             -0.16              0.               -0.39
 -0.07              0.0025            0.0256            0.
  0.1521            0.0049            0.95122942450071  0.85214378896621
  1.                0.67705687449816  0.93239381990595  0.99875026039497
  0.98722728337563  1.                0.92490905985731  0.99755100025328
  1.              ]


In [94]:

def get_params_ridge(params):
    # params["solver"] = 'lsqr' 
    params["fit_intercept"] = False
    return params


def bo_tune_ridge(alpha):

    params = get_params_ridge({"alpha": alpha})
    mdl = Ridge(alpha=alpha, fit_intercept=False)

    print(f"Params: alpha={alpha}")
    scores = cross_val_score(mdl, X_train, y_train, cv=10, n_jobs=5, scoring="neg_root_mean_squared_error")
    print(scores, scores.mean())
    # print("recall", recall)

    return scores.mean()

parameter_range_ridge = {
    "alpha": (0, 10)
}

ridge = Ridge, bo_tune_ridge, parameter_range_ridge, get_params_ridge


def get_params_elastic(params):
    # params["solver"] = 'lsqr' 
    params["fit_intercept"] = False
    return params

def bo_tune_elastic(alpha, l1_ratio):

    mdl = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=False)

    print(f"Params: alpha={alpha}")
    scores = cross_val_score(mdl, X_train, y_train, cv=10, n_jobs=5, scoring="neg_root_mean_squared_error")
    
    print(scores, scores.mean())
    # print("recall", recall)

    return scores.mean()

parameter_range_elastic = {
    "l1_ratio": (0, 1),
    "alpha": (0, 10)
}

elastic = ElasticNet, bo_tune_elastic, parameter_range_elastic, get_params_elastic


def get_params_lasso(params):
    # params["solver"] = 'lsqr' 
    params["fit_intercept"] = False
    return params

def bo_tune_lasso(alpha):

    mdl = Lasso(alpha=alpha, fit_intercept=False)

    print(f"Params: alpha={alpha}")
    scores = cross_val_score(mdl, X_train, y_train, cv=10, n_jobs=5, scoring="neg_root_mean_squared_error")
    
    print(scores, scores.mean())
    # print("recall", recall)

    return scores.mean()

parameter_range_lasso = {
    "alpha": (0, 10)
}

lasso = Lasso, bo_tune_lasso, parameter_range_lasso, get_params_lasso

# def bo_tune_lin_reg():

#     mdl = LinearRegression()

#     print(f"Params: None")
#     scores = cross_val_score(mdl, X_train, y_train, cv=10, n_jobs=5, scoring="neg_root_mean_squared_error")
#     print(scores, scores.mean())
#     # print("recall", recall)

#     return scores.mean()

# parameter_range_lin_reg = {
# }

# lin_reg = LinearRegression, bo_tune_lin_reg, parameter_range_lin_reg

to_test = [elastic, ridge, lasso] # [ridge, lasso]

In [95]:
best_mdls = []
# best_mdls.append(LinearRegression().fit(X_train, y_train))
for mdl, tune_func, param_ranges, to_params in to_test:
  bo = BayesianOptimization(tune_func, param_ranges)
    #performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
  bo.maximize(n_iter=30, init_points=20, acq='ei')

  #Extracting the best parameters
  params = bo.max['params']
  print(params)

  best_score = bo.max["target"]
  print("best score ", best_score)
  ##  best alpha = 5.2
  # params = {"alpha": 5.2}
  params = to_params(params)
  model = mdl(**params).fit(X_train, y_train)
  
  best_mdls.append(model)



|   iter    |  target   |   alpha   | l1_ratio  |
-------------------------------------------------
Params: alpha=6.494677988223141
[-5.1065276139152  -4.93733751878385 -5.19207011815992 -4.82924196110759
 -5.55754708477611 -4.73393897844647 -5.29301123339129 -4.96515067603681
 -4.50819159937018 -4.80692947409641] -4.992994625808385
|  1        | -4.993    |  6.495    |  0.6216   |
Params: alpha=6.211436875021333
[-4.86036302473819 -4.69950788055602 -4.95115238637324 -4.58221204743548
 -5.31335090338755 -4.50042186554946 -5.0497917236573  -4.72587379786294
 -4.27387319492674 -4.56993412398947] -4.752648094847638
|  2        | -4.753    |  6.211    |  0.5966   |
Params: alpha=6.887279570219631
[-4.27656957874551 -4.14425857274597 -4.38310491332497 -4.00030106764434
 -4.7324559144425  -3.95740576721139 -4.47522895429794 -4.16751896308662
 -3.73571429369697 -4.01696657111403] -4.188952459631024
|  3        | -4.189    |  6.887    |  0.3544   |
Params: alpha=8.347135275469865
[-6.757680393

/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.000e+03, tolerance: 2.490e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.987e+03, tolerance: 2.505e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklea

[-2.14948800793354 -2.23712343586307 -2.32499247208659 -1.89332100935667
 -2.58335664027109 -2.22176015272511 -2.37846460505768 -2.24233810260829
 -2.02965894351646 -2.14116646206831] -2.2201669831486806
|  23       | -2.22     |  2.034    |  0.0      |
Params: alpha=1.4019242216112016
[-2.3936886398311  -2.40370082346321 -2.53166079918402 -2.0995292291593
 -2.83022554958272 -2.36490301106977 -2.61092592436755 -2.42719166543871
 -2.14152836026344 -2.29905283129594] -2.4102406833655747
|  24       | -2.41     |  1.402    |  0.9964   |
Params: alpha=0.3995519983089668


/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.674e+00, tolerance: 2.490e+00
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.100e+00, tolerance: 2.505e+00
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.642e+

[-1.88051611648014 -2.03279414950347 -2.03973883958416 -1.62404622157811
 -2.22772118144162 -2.12469356140427 -2.07177067178218 -2.05348415564995
 -1.96588432191039 -1.97891157416472] -1.9999560793499036
|  25       | -2.0      |  0.3996   |  0.9998   |
Params: alpha=0.581716979863486


...

[-1.86184672671543 -1.9863033426309  -1.96825046172105 -1.57981774133855
 -2.06564806114848 -2.14414610112612 -1.94538785311641 -1.98671105141633
 -2.0680955878604  -2.02653849780083] -1.9632745424874483
|  22       | -1.963    |  9.521e-0 |
Params: alpha=0.0016903603539952794


/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.207e+00, tolerance: 2.490e+00
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.695e+00, tolerance: 2.505e+00
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.225e+

[-1.81518382178869 -1.99025267199247 -1.96042759002622 -1.57621241279788
 -2.04625365930678 -2.13473035387006 -1.93975898325135 -1.98469522248001
 -2.06719244709772 -1.99089317243147] -1.9505600335042648
|  23       | -1.951    |  0.00169  |
Params: alpha=9.101745382711357e-05


/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.247e+02, tolerance: 2.505e+00
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.423e+02, tolerance: 2.485e+00
  model = cd_fast.enet_coordinate_descent(
/home/nick/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.316e+

[-1.86204535895474 -1.98630673148296 -1.968305967279   -1.57998610649086
 -2.06579596175598 -2.14424114395652 -1.94533367248387 -1.98676025828498
 -2.06811278082629 -2.02674535313018] -1.9633633334645384
|  24       | -1.963    |  9.102e-0 |
Params: alpha=0.07871350858640103
[-1.7935827378408  -1.99366908115162 -1.96191847642937 -1.57938772323239
 -2.07624195802813 -2.12843173111334 -1.9590549366135  -2.00937835779284
 -2.02556169164771 -1.97574499532221] -1.9502971689171926
|  25       | -1.95     |  0.07871  |
Params: alpha=0.0800788747957848
[-1.79382135879668 -1.99392418216822 -1.96219289367383 -1.57941402471966
 -2.07679622985925 -2.1282814265674  -1.95941994997395 -2.00944717594939
 -2.02511549655121 -1.97556996936733] -1.950398270762691
|  26       | -1.95     |  0.08008  |
Params: alpha=0.0832857147617505
[-1.79438926782568 -1.99452867534483 -1.96284142593156 -1.57948434890804
 -2.07810302370596 -2.12793393438006 -1.96028676411553 -2.00962216015628
 -2.02407424853241 -1.9751672

In [96]:
from sklearn.metrics import mean_squared_error

best_val_error = 100000000
for mdl in best_mdls:

    print(f"MODEL {mdl}:")
    score = mdl.score(X_test, y_test)

    print(f"   SCORE: {score}")
    #Looking at the classification report
    print(f"COEFFS: {mdl.coef_}")

    y_pred_test = mdl.predict(X_test)
    y_pred_train = mdl.predict(X_train)

    train_error = mean_squared_error(y_train, y_pred_train, squared=False)
    validation_error = mean_squared_error(y_test, y_pred_test, squared=False)

    print("train_error", train_error)
    print("validation_error", validation_error)

    if(validation_error < best_val_error):
        best_val_error = validation_error
        best_model = mdl
    # print(y_pred_test)


MODEL ElasticNet(alpha=0.02294577885611937, fit_intercept=False,
           l1_ratio=0.9455323891912856):
   SCORE: 0.004014170559650698
COEFFS: [ 0.               -0.               -0.                0.
  0.               -0.                0.                0.
 -0.               -0.               -0.               -1.58558198763499
 -1.95110413657762 -0.               -0.               -0.4573256021322
 -1.63323856660041 -0.75536376716751 -0.               -0.03987855495858
 -0.1015870972053 ]
train_error 1.9435863913710139
validation_error 1.98982694716749
MODEL Ridge(alpha=5.43376698039975, fit_intercept=False):
   SCORE: -0.0058187169325116805
COEFFS: [ 0.40817151722712 -0.4794794846399  -0.55585553092405  0.34366070379567
  0.3090944365502  -0.35251216774685  0.08117237757756  0.10726519958863
 -0.23130027961696 -0.06172580453536 -0.36577225119155 -1.03261322426938
 -1.09912308133646 -0.35855898260309 -0.31069130358285 -0.41720311284835
 -0.63270023172964 -0.64564955614287 -0.479

In [97]:


save_submission(best_model.coef_)


In [98]:
print(X_test)

[array([-2.00000000000000e-01,  1.20000000000000e-01,
       -1.00000000000000e-01, -4.00000000000000e-01,
       -3.00000000000000e-02,  4.00000000000000e-02,
        1.44000000000000e-02,  1.00000000000000e-02,
        1.60000000000000e-01,  9.00000000000002e-04,
        8.18730753077982e-01,  1.12749685157938e+00,
        9.04837418035960e-01,  6.70320046035639e-01,
        9.70445533548508e-01,  9.80066577841242e-01,
        9.92808635853866e-01,  9.95004165278026e-01,
        9.21060994002885e-01,  9.99550033748988e-01,
        1.00000000000000e+00]), array([-2.30000000000000e-01, -1.00000000000000e-02,
       -5.00000000000000e-02, -1.30000000000000e-01,
       -5.00000000000000e-02,  5.29000000000000e-02,
        1.00000000000000e-04,  2.50000000000000e-03,
        1.69000000000000e-02,  2.50000000000000e-03,
        7.94533602503334e-01,  9.90049833749168e-01,
        9.51229424500714e-01,  8.78095430920561e-01,
        9.51229424500714e-01,  9.73666395005375e-01,
        9.999